In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
data = pd.read_csv(r"C:\Users\sadha\Downloads\digit-recognizer\train.csv")


In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[4 4 4 ... 4 4 0] [7 7 3 ... 7 6 9]
0.09865853658536586
Iteration:  10
[7 8 3 ... 6 6 6] [7 7 3 ... 7 6 9]
0.19617073170731708
Iteration:  20
[7 7 3 ... 6 6 6] [7 7 3 ... 7 6 9]
0.2592926829268293
Iteration:  30
[7 7 3 ... 6 6 6] [7 7 3 ... 7 6 9]
0.3444634146341463
Iteration:  40
[7 7 3 ... 7 6 6] [7 7 3 ... 7 6 9]
0.41448780487804876
Iteration:  50
[7 7 3 ... 7 6 6] [7 7 3 ... 7 6 9]
0.4673658536585366
Iteration:  60
[7 7 3 ... 7 3 6] [7 7 3 ... 7 6 9]
0.5115853658536585
Iteration:  70
[7 7 3 ... 7 3 9] [7 7 3 ... 7 6 9]
0.5460487804878049
Iteration:  80
[7 7 3 ... 7 3 9] [7 7 3 ... 7 6 9]
0.579
Iteration:  90
[7 7 3 ... 7 3 9] [7 7 3 ... 7 6 9]
0.6059268292682927
Iteration:  100
[7 7 3 ... 7 3 9] [7 7 3 ... 7 6 9]
0.63
Iteration:  110
[7 7 3 ... 7 3 9] [7 7 3 ... 7 6 9]
0.6515853658536586
Iteration:  120
[7 7 3 ... 7 8 9] [7 7 3 ... 7 6 9]
0.6704146341463415
Iteration:  130
[7 7 3 ... 7 8 9] [7 7 3 ... 7 6 9]
0.6865121951219513
Iteration:  140
[7 7 3 ... 7 8 9] [7 7 3 

In [7]:
_, _, _, A2_dev = forward_prop(W1, b1, W2, b2, X_dev)
dev_acc = get_accuracy(get_predictions(A2_dev), Y_dev)
print(f"Development set accuracy: {dev_acc:.4f}")

[1 4 0 8 1 8 8 1 6 0 3 6 8 8 9 2 1 4 7 5 1 1 1 0 5 2 6 6 2 6 0 1 0 4 1 8 9
 1 1 8 7 8 2 8 0 9 0 9 6 1 2 3 0 4 2 9 1 5 2 6 6 1 8 1 8 2 7 6 9 3 4 2 9 1
 6 0 5 2 2 6 7 5 3 7 5 9 7 3 9 0 0 7 4 9 9 2 5 7 4 9 7 7 0 8 7 6 6 1 3 5 6
 8 3 1 8 8 0 1 3 0 0 2 0 6 1 7 0 9 2 2 4 5 2 1 4 7 2 0 5 0 5 8 8 7 9 7 1 3
 6 3 8 4 0 0 1 2 1 3 0 2 4 5 3 1 3 5 3 2 8 2 9 1 3 5 6 6 1 4 0 8 7 5 1 4 7
 8 1 3 8 6 0 1 8 7 4 4 0 3 6 8 5 8 9 0 4 1 2 5 4 1 4 1 4 8 7 3 6 7 4 6 6 3
 3 4 2 1 7 1 9 6 9 2 3 3 2 3 1 2 3 0 7 3 6 7 4 7 2 6 8 7 3 0 2 0 4 1 9 1 4
 4 0 8 3 2 2 3 6 5 2 7 3 7 6 4 1 3 3 8 3 4 7 5 8 3 5 5 4 8 8 9 1 9 9 9 1 0
 9 4 0 9 5 7 2 5 3 8 5 8 6 2 0 5 5 3 8 6 4 2 6 7 3 7 4 4 1 0 5 1 6 5 9 2 1
 3 1 2 6 2 7 3 4 5 4 7 1 9 1 6 5 2 6 9 7 5 4 2 7 7 5 4 4 4 6 6 0 5 2 7 4 2
 3 2 2 5 0 3 3 7 7 3 0 0 5 7 5 4 3 1 7 8 2 5 3 0 7 3 5 1 7 1 5 0 1 4 6 9 0
 7 3 8 9 4 2 8 4 6 3 7 1 5 0 9 3 8 5 1 2 7 2 2 1 6 6 0 2 3 1 1 2 1 3 1 2 8
 8 9 6 2 7 9 1 2 4 2 2 2 6 2 4 0 4 1 0 0 6 6 1 2 1 3 3 9 1 4 2 7 3 1 8 9 2
 0 7 0 2 8 8 7 5 0 0 0 6 